In [1]:
# Python equivalent: create pseudo_fragments from .h5mu
import muon as mu
import numpy as np
import scipy.sparse as sp
from pathlib import Path

## reading process

h5mu_path = "data/HT.h5mu"
out_path = Path("data/HT_pseudo_fragments.tsv")
out_path.parent.mkdir(parents=True, exist_ok=True)
# Load MuData
mdata = mu.read_h5mu(h5mu_path)

/home/xli/github/PSC/ShinyCell2_multiomic/.venv/lib/python3.13/site-packages/muon/_core/preproc.py:31: FutureWarning: `__version__` is deprecated, use `importlib.metadata.version('scanpy')` instead
  if Version(scanpy.__version__) < Version("1.10"):
/home/xli/github/PSC/ShinyCell2_multiomic/.venv/lib/python3.13/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/xli/github/PSC/ShinyCell2_multiomic/.venv/lib/python3.13/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, w

In [2]:
mdata.mod

MuData
├─ atac_cell_by_bin AnnData (115337 x 6062095)
└─ atac_cell_by_gene AnnData (115337 x 24919)

In [3]:
# Pick ATAC-by-bin modality. Adjust the key if needed.
# Common keys: "atac_cell_by_bin", "atac", "chrombins"
candidate_keys = ["atac_cell_by_bin"]#, "atac", "chrombins"]
for k in candidate_keys:
    if k in mdata.mod:
        ad = mdata.mod[k]
        break
else:
    # fallback to the first modality
    ad = next(iter(mdata.mod.values()))

# Get sparse count matrix
if "counts" in ad.layers:
    mat = ad.layers["counts"]
else:
    mat = ad.X
if not sp.issparse(mat):
    mat = sp.csr_matrix(mat)

In [4]:
# Row/col names
bins = np.array(ad.var_names)  # rows
cells = np.array(ad.obs_names)  # cols

# split into at least 3 fields; pad if shorter
parts = np.char.split(bins.astype(str))
#parts = np.array([p if len(p) >= 3 else (p + ["0"]*(3-len(p))) for p in parts], dtype=object)

def to_int(x):
    try:
        return int(float(x))
    except Exception:
        # fallback: extract first number from the string
        m = re.search(r'[+-]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][+-]?\d+)?', str(x))
        return int(float(m.group(0))) if m else 0
    
chrom = np.array([p[0] for p in parts], dtype=object)
start = np.array([to_int(p[2]) for p in parts], dtype=np.int64)
end = start + 499

In [5]:
# Iterate over nonzeros
out_path = Path("data/HT_pseudo_fragments.tsv")
coo = mat.tocoo()

In [8]:
## final output
limit = 10000000
n_written = 0
stop = False
with out_path.open("a") as f:
    for c, r, v in zip(coo.row, coo.col, coo.data):
        v = int(v)
        if v <= 0:
            continue
        else:
            f.write(f"{chrom[r]}\t{start[r]}\t{start[r]+1}\t{cells[c]}\t{v}\n")
            n_written += 1
            if n_written >= limit:
                    stop = True
                    break
        if stop:
            break

print(f"Wrote {n_written} fragments to {out_path}")

Wrote 10000000 fragments to data/HT_pseudo_fragments.tsv
